In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)
matplotlib 3.1.1
numpy 1.17.2
pandas 0.25.3
sklearn 0.21.3
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
# tf.function and auto-graph
# python 语法
def scaled_elu(z, scale=1.0, alpha=1.0):
    ''' 实现 z >= 0 ? scale * z : scale * alpha * tf.nn.elu(z)  # tf.where实现三目运算符'''
    is_positive = tf.greater_equal(z, 0.0) # z >= 0
    return scale * tf.where(is_positive, z, alpha * tf.nn.elu(z))

print(scaled_elu(tf.constant(-3.6)))
print(scaled_elu(tf.constant([-3.6, -2.5])))

tf.Tensor(-0.9726763, shape=(), dtype=float32)
tf.Tensor([-0.9726763 -0.917915 ], shape=(2,), dtype=float32)


In [3]:
# 通过tf.function将python语法的自定义方法转成TF语法
scaled_elu_tf = tf.function(scaled_elu) 
print(scaled_elu_tf(tf.constant(-3.6)))
print(scaled_elu_tf(tf.constant([-3.6, -2.5])))

tf.Tensor(-0.9726763, shape=(), dtype=float32)
tf.Tensor([-0.9726763 -0.917915 ], shape=(2,), dtype=float32)


In [4]:
print(scaled_elu_tf.python_function)

<function scaled_elu at 0x7fd8dd60bf28>


In [5]:
print(scaled_elu_tf.python_function is scaled_elu)

True


In [6]:
# 转成Tensorflow中的图 运算速度更快
%timeit scaled_elu(tf.random.normal((1000, 1000)))
%timeit scaled_elu_tf(tf.random.normal((1000, 1000))) # 运算速度快

15.3 ms ± 1.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
15.1 ms ± 1.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
# @tf.function是另外一种将python语法转为tf图结构的方法

@tf.function
def converge_to_2(n_iters):
    # 1 + 1/2 + 1/2^2 + ...+1/2^n
    total = tf.constant(0.)
    increment = tf.constant(1.)
    for _ in range(n_iters):
        total += increment
        increment /= 2.0
    return total

print(converge_to_2(100)) 

tf.Tensor(2.0, shape=(), dtype=float32)


In [8]:
def display_tf_code(func):
    '''将python语法转为TF语法结构并显示其代码'''
    code = tf.autograph.to_code(func)
    from IPython.display import display, Markdown
    display(Markdown('```python\n{}\n```'.format(code)))

In [9]:
display_tf_code(scaled_elu)

```python
def tf__scaled_elu(z, scale=None, alpha=None):
  """ 实现 z >= 0 ? scale * z : scale * alpha * tf.nn.elu(z)  # tf.where实现三目运算符"""
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('scaled_elu', 'scaled_elu_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as scaled_elu_scope:
    is_positive = ag__.converted_call(tf.greater_equal, scaled_elu_scope.callopts, (z, 0.0), None, scaled_elu_scope)
    do_return = True
    retval_ = scaled_elu_scope.mark_return_value(scale * ag__.converted_call(tf.where, scaled_elu_scope.callopts, (is_positive, z, alpha * ag__.converted_call(tf.nn.elu, scaled_elu_scope.callopts, (z,), None, scaled_elu_scope)), None, scaled_elu_scope))
  do_return,
  return ag__.retval(retval_)

```

In [10]:
display_tf_code(converge_to_2)  # ？？？？

ConversionError: converting <tensorflow.python.eager.def_function.Function object at 0x7fd893c5ba58>: AttributeError: 'Function' object has no attribute '__code__'

In [11]:
@tf.function
def add_99():
    var = tf.Variable(0.)
    return var.assign_add(99)  # Variable += 但不能直接用符号

try:
    print(add_99())
except ValueError as ex:
    print(ex)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
in converted code:

    <ipython-input-11-75430022aad5>:3 add_99  *
        var = tf.Variable(0.)
    /home/kdd/Env/TF2.0/lib/python3.6/site-packages/tensorflow_core/python/ops/variables.py:260 __call__
        return cls._variable_v2_call(*args, **kwargs)
    /home/kdd/Env/TF2.0/lib/python3.6/site-packages/tensorflow_core/python/ops/variables.py:254 _variable_v2_call
        shape=shape)
    /home/kdd/Env/TF2.0/lib/python3.6/site-packages/tensorflow_core/python/ops/variables.py:65 getter
        return captured_getter(captured_previous, **kwargs)
    /home/kdd/Env/TF2.0/lib/python3.6/site-packages/tensorflow_core/python/eager/def_function.py:413 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.function-decorated function tried to create variables on non-first call.



In [12]:
var = tf.Variable(0.) # 定义在tf.function外部才能被初始化
@tf.function
def add_99():  
    return var.assign_add(99)

print(add_99())

tf.Tensor(99.0, shape=(), dtype=float32)


In [13]:
@tf.function
def cube(z):
    return tf.pow(z, 3)

print(cube(tf.constant([1., 2., 3.])))
print(cube(tf.constant([1, 2, 3])))

tf.Tensor([ 1.  8. 27.], shape=(3,), dtype=float32)
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


In [14]:
# 通过函数签名input_signature参数限定输入的数据类型
@tf.function(input_signature=[tf.TensorSpec([None], tf.int32, name='x')])
def cube(z):
    return tf.pow(z, 3)

try:
    print(cube(tf.constant([1., 2., 3.])))
except ValueError as ex:
    print(ex)
    
print(cube(tf.constant([1, 2, 3])))

Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name='x'))
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


In [15]:
# @tf.function     py function ->tf graph
# get_concrete_function -> add input signature ->SaveModel
cube_func_int32 = cube.get_concrete_function(tf.TensorSpec([None], tf.int32))

In [16]:
print(cube_func_int32)

In [17]:
print(cube_func_int32 is cube.get_concrete_function(tf.TensorSpec([5], tf.int32)))
print(cube_func_int32 is cube.get_concrete_function(tf.constant([1,2,3])))

True
True


In [18]:
cube_func_int32.graph

In [19]:
cube_func_int32.graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'Pow/y' type=Const>,
 <tf.Operation 'Pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [20]:
pow_op = cube_func_int32.graph.get_operations()[2]

In [21]:
print(pow_op)

name: "Pow"
op: "Pow"
input: "x"
input: "Pow/y"
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}



In [22]:
print(list(pow_op.inputs))
print(list(pow_op.outputs))

[<tf.Tensor 'x:0' shape=(None,) dtype=int32>, <tf.Tensor 'Pow/y:0' shape=() dtype=int32>]
[<tf.Tensor 'Pow:0' shape=(None,) dtype=int32>]


In [23]:
cube_func_int32.graph.get_operation_by_name('x')

<tf.Operation 'x' type=Placeholder>

In [24]:
cube_func_int32.graph.get_tensor_by_name('x:0')

<tf.Tensor 'x:0' shape=(None,) dtype=int32>

In [25]:
cube_func_int32.graph.as_graph_def()

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "Pow/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 3
      }
    }
  }
}
node {
  name: "Pow"
  op: "Pow"
  input: "x"
  input: "Pow/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "Identity"
  op: "Identity"
  input: "Pow"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
versions {
  producer: 119
}